In [28]:
#before, I run in conda pip install -U pyvisa
import skrf as rf
import pyvisa
import time #in case some commands take longer for the vna to process, I can forcefully make python wait
from datetime import datetime,timezone,timedelta
print('Using pyvisa ',pyvisa.__version__)
rm = pyvisa.ResourceManager()#after importing visa, we create a resource manager. ResourceManager will provide you with the appropriate class. 
vna = rm.open_resource('TCPIP::169.254.156.254::INSTR') #here, my TCPIP is given by the VNA IP address.open_resource has given you an instance of GPIBInstrument class (a subclass of the more generic Resource
"""
For most instruments, you actually need to properly configure the instrument so that it understands the message sent by the computer (in particular how to identifies the end of the commands) and so that computer knows
when the instrument is done talking. If you don’t you are likely to see a VisaIOError reporting a timeout.
"""
vna.read_termination = '\n' #VNA manual says this is the termination character. Like this, the PC knows if the VNA has stopped talking when this character appears. I confirmed this is the character in several ways
vna.write_termination = '\n' #now, by default, what I write will be added to a \n, so I do not need to add it. I confirmed this is the character in several ways.
#vna.baud_rate = 57600 #baud rate refers to the rate at which the serial connection transfer data. 9600 by default, VNA manual does not specify
print('Connection established to ',vna.query('*IDN?')) #standard GPIB message for “what are you?” or – in some cases – “what’s on your display at the moment?”. query is a short form for a write operation to send a message, followed by a read.
print('VNA programming language set to ',vna.query('LANG?'))

Using pyvisa  1.12.0
Connection established to  ANRITSU,37397D,1813864,V2022.6.1
VNA programming language set to  LIGHT


In [3]:
#setting and checking the adapter length for adapter removal with auto cal kit
#vna.write("ACADPL 0") #meters
#vna.query("ACADPL?")

In [18]:
start_freq_setting = 18e9 #Hz
end_freq_setting = 20e9 #Hz
number_points_setting = 2001 #Delta F should not have many decimal places, otherwise numerical errors can occur
power_setting = 0 #dBm. Can be high as long as I am not using any active components such as an amplifier
IFBW_setting = 300 #Hz. Refers to the resolution bandwidth of the device. Low enough to reduce noise without sacrificing measuring time too much
# Set the frequency range
vna.write(':SENSe1:FREQuency:STARt {0};:SENSe1:FREQuency:STOP {1}'.format(start_freq_setting,end_freq_setting))
#set the number of points
vna.write(":SENS1:SWE:POIN {0}".format(number_points_setting))
#setting the power level and the IFBW
vna.write('PWR {0};SENS:BWID {1}'.format(power_setting,IFBW_setting))


20

In [23]:
# Query the frequency range to confirm the settings
start_freq_read = vna.query(':SENS1:FREQ:STAR?',delay=2) #Some instruments may be slow in answering and may require you to either increase the timeout or specify a delay between the write and read operation.
                                                      #wait 100 ms after writing before reading.
stop_freq_read = vna.query(':SENS1:FREQ:STOP?',delay=2)
number_points_read = vna.query(':SENS1:SWE:POIN?',delay=2)
IFBW_read = vna.query('SENS:BWID?',delay=2)
power_read = vna.query('PWR?',delay=2)


In [24]:
print('Frequency range:', start_freq_read.strip(), 'Hz to', stop_freq_read.strip(),' Hz with ',number_points_read,' points. \n Output power set to ',power_read,' dBm and IFBW of ',IFBW_read,' Hz')

Frequency range: 1.80000000000E+10 Hz to 2.00000000000E+10  Hz with  2084  points. 
 Output power set to   0.00000000000E+00  dBm and IFBW of   3.00000000000E+02  Hz


In [29]:
def autoscale():
    try:
        #To autoscale the traces. Do this everytime before saving the data. It can improve the resolution.
        vna.write('CHAN1') #to select trace 1 as active trace
        vna.write("ASC") #to autoscale active trace

        vna.write('CHAN2')
        vna.write("ASC")

        vna.write('CHAN3')
        vna.write("ASC")

        vna.write('CHAN4')
        vna.write("ASC")

        return None
    except:
        return print('Could not autoscale the traces')
    

def datetime_name():
    return datetime.now(timezone.utc).strftime("%y%m%d%H%M%S")

In [30]:
storing_folder_in_VNA = "C:/Users/VectorStarUser/Desktop/Juan/continous_measurement/"
filename_in_VNA = "_VNAp1_SP_VNAp2_port6.s2p"
datetime_initial = datetime.now()
taking_data_for = 48 #hours
delta_t = 40 #seconds, time between samples

autoscale() #autoscaling all the traces before starting

while True:
    #taking data for a limited amount of time, given by 
    if datetime.now() - datetime_initial < timedelta(hours = taking_data_for):
        #autoscale() #autoscale all traces each hour in case major changes have happened
        
        #first we take a single sweep
        vna.write("SENSe:HOLD:FUNCtion SINGLE")
        finished_save = vna.query("*OPC?")

        #Then we store the file on the VNA
        vna.write(":MMEMory:STORe " + "'" + storing_folder_in_VNA + datetime_name() +filename_in_VNA + "'")
        finished_save = vna.query("*OPC?")

        #now we wait some seconds to take data again
        time.sleep(delta_t) # sleep between iterations
    elif datetime.now() - datetime_initial >= timedelta(hours = taking_data_for):
        #After data taking, finish the remote connection
        try:
            # Send a command to return the VNA to local mode
            vna.write("SENSe:HOLD:FUNCtion CONT")
            vna.write("RTL")
            #Close the device on PyVISA
            vna.close()
            print('Safe to manually operate the VNA again, remote control turned OFF')
            break
        except:
            print('were not able to go back to local mode. You can also exit remote mode by pressing ctrol-> in the device panel')

Safe to manually operate the VNA again, remote control turned OFF


In [8]:
try:
    # Send a command to return the VNA to local mode
    vna.write("SENSe:HOLD:FUNCtion CONT")
    vna.write("RTL")
    #Close the device on PyVISA
    vna.close()
    print('Safe to manually operate the VNA again, remote control turned OFF')
except:
    print('were not able to go back to local mode. You can also exit remote mode by pressing ctrol-> in the device panel')

were not able to go back to local mode. You can also exit remote mode by pressing ctrol-> in the device panel
